<a href="https://colab.research.google.com/github/namanphy/END2/blob/main/S4/Basic_LSTM_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:05<00:00, 15.7MB/s]


In [12]:
TEXT

In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [13]:
print(vars(train_data.examples[0]))

{'text': ['Before', 'Cujo', ',', 'there', 'was', 'Lucky', 'the', 'devil', 'dog', '.', 'In', '1978,on', 'Halloween', 'night', 'the', 'movie"Devil', 'Dog', ',', 'The', 'Hound', 'of', 'Hell', '"', 'premiered', '.', 'A', 'story', 'of', 'a', 'family', 'getting', 'a', 'new', 'puppy', '(', 'from', 'a', 'farmer', 'who', 'just', 'happen', 'to', 'be', 'in', 'the', 'neighborhood', 'selling', 'fruits', 'and', 'vegetables', ')', 'because', 'their', 'dog', 'Skipper', 'was', 'killed', '.', 'Coencidence', '?', 'Everyone', 'loves', 'the', 'new', 'dog', ',', 'but', 'there', 'is', 'something', 'strange', 'about', 'him', '.', '<', 'br', '/><br', '/>It', 'is', "n't", 'long', 'until', 'the', 'father', 'Mike', 'Barry(Richard', 'Crenna', ',', 'First', 'Blood)starts', 'to', 'notice', '.', 'His', 'wife', 'Betty(Yvette', 'Mimieux', ',', 'Where', 'The', 'Boys', 'Are', ',', 'Jackson', 'County', 'Jail', ',', 'Snowbeast)is', 'different', 'and', 'his', 'kids', 'Charlie', 'and', 'Bonnie(Ike', 'Eisenman', ',', 'Witch',

In [15]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [17]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 12250
Number of validation examples: 5250
Number of testing examples: 25000


In [18]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [19]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [20]:
# Most frequent words
print(TEXT.vocab.freqs.most_common(20))

[('the', 141851), (',', 134260), ('.', 116080), ('and', 76309), ('a', 76009), ('of', 70244), ('to', 65460), ('is', 53267), ('in', 42898), ('I', 38163), ('it', 37518), ('that', 34589), ('"', 30789), ("'s", 30527), ('this', 29715), ('-', 25762), ('/><br', 24684), ('was', 24671), ('as', 21302), ('with', 20995)]


In [21]:
# Interger to strings

print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [22]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


## Creating Iterators

In [36]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

## Building Model


### Using LSTM's hidden state to pass through FC for final output.

In [37]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        self.fc1 = nn.Sequential(
            nn.Linear(hidden_dim, 128),
            nn.ReLU()
        )
        self.fc2 = nn.Linear(128, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]

        output, (hidden, cell_state) = self.lstm(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]

        # print('ALRIGHT TILL HERE')
        # print(hidden)

        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        return self.fc2(self.fc1(hidden.squeeze(0)))

In [38]:
# Specifying Dimensions
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

# Model Instantiation
model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [39]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,899,817 trainable parameters


In [40]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [41]:
criterion = nn.BCEWithLogitsLoss()

In [42]:
model = model.to(device)
criterion = criterion.to(device)

In [43]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

## Train / Test loop

In [44]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [45]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [46]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Training

In [47]:
N_EPOCHS = 40

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f' | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f' | Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 50s
 | Train Loss: 0.694 | Train Acc: 50.00%
 | Val. Loss: 0.693 |  Val. Acc: 50.64%
Epoch: 02 | Epoch Time: 0m 51s
 | Train Loss: 0.693 | Train Acc: 50.21%
 | Val. Loss: 0.693 |  Val. Acc: 50.58%
Epoch: 03 | Epoch Time: 0m 50s
 | Train Loss: 0.693 | Train Acc: 50.31%
 | Val. Loss: 0.693 |  Val. Acc: 49.94%
Epoch: 04 | Epoch Time: 0m 50s
 | Train Loss: 0.693 | Train Acc: 50.53%
 | Val. Loss: 0.693 |  Val. Acc: 49.94%
Epoch: 05 | Epoch Time: 0m 50s
 | Train Loss: 0.693 | Train Acc: 50.56%
 | Val. Loss: 0.696 |  Val. Acc: 50.79%
Epoch: 06 | Epoch Time: 0m 50s
 | Train Loss: 0.692 | Train Acc: 50.73%
 | Val. Loss: 0.700 |  Val. Acc: 49.57%
Epoch: 07 | Epoch Time: 0m 50s
 | Train Loss: 0.692 | Train Acc: 50.85%
 | Val. Loss: 0.695 |  Val. Acc: 50.53%
Epoch: 08 | Epoch Time: 0m 50s
 | Train Loss: 0.691 | Train Acc: 50.95%
 | Val. Loss: 0.699 |  Val. Acc: 50.73%
Epoch: 09 | Epoch Time: 0m 51s
 | Train Loss: 0.689 | Train Acc: 50.54%
 | Val. Loss: 0.706 |  Val. Acc:

In [48]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.652 | Test Acc: 73.10%


### Better accuracy over RNN - if passing the final hidden state with two fully connected layers.

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.693 | Test Acc: 52.39%


### **Using Adam as the optimizer - helped in reaching a high 99% accuracy on training data. And 79% on validation data.**

### **On the Test data - accuracy reached 73%.**

### Conclusion - The model is clearly overfitting and converging very slowly with taking a much time.